# Actividad M1. Limpiar una habitación sucia con agentes
Santiago Álvarez Valdivia **A01640172**

Héctor Manuel Cárdenas Yáñez **A01634615**

Jueves, 18 de agosto de 2022

## Problema Propuesto
El problema propuesto por el profesor consiste en crear un script de python que modele una habitación con celdas sucias, en donde hay un número de robots limpiadores, que pueden limpiar una celda adyacente o moverse en una dirección aleatoria. La información que el programa debe recibir como parámetros es:
- El tamaño de la habitación (MxN)
- Número de limpiadores
- Porcentaje de celdas inicialmente sucias
- Pasos máximos de ejecución

## Código

Primero importamos las librerías de Python necesarias para correr el script

In [14]:
import agentpy as ap
import random
import matplotlib.pyplot as plt
import IPython

Definimos los movimientos que puede realizar cada limpiador, en este caso 8, asimismo, el valor de los parámetros de ejecución

In [15]:
MOVES = [(-1,-1),(-1,0),(-1,1), (0,-1), (0, 1), (1, -1), (1, 0), (1,1)]
parameters = {
    'steps': 200,
    'cleaners': 24,
    'M': 50,
    'N': 40,
    'dirtyPercentage': 0.3
}

Definimos el modelo, el graficador y ejecutamos el programa

In [16]:
class DirtyRoom(ap.Model):
    
    def setup(self): #Función inicial del modelo
        self.cleaners = ap.AgentList(self, self.p['cleaners'])
        self.dirty = ap.AgentList(self, int(self.p['M']*self.p['N']*self.p['dirtyPercentage']))

        self.room = ap.Grid(self, [self.p['M'], self.p['N']], track_empty=True)
        self.room.add_agents(self.cleaners, [(1, 1)]*self.p['cleaners'])
        self.room.add_agents(self.dirty, random=True, empty=True)

        #Condition 0: Robot, 1: Trash, 2: Cleaned Trash

        self.cleaners.type_agent = 0
        self.dirty.type_agent = 1
    
    def step(self):
        
        robots = self.cleaners
        
        #Para cada limpiador que haya
        for cleaner in robots:
            
            #Limpiar una de las celdas sucias vecinas
            for closeCell in self.room.neighbors(cleaner, distance=1):
                if closeCell.type_agent == 1:
                    closeCell.type_agent = 2
                    break
            else:
                #O si no hay ninguna, moverse
                self.room.move_by(cleaner, random.choice(MOVES))
        
        trash_pending = self.dirty.select(self.dirty.type_agent == 1)
        
        if len(trash_pending) == 0:
            self.stop()
    
    def end(self):
        cleaned = len(self.dirty.select(self.dirty.type_agent == 2))
        self.report('Percentage of cleaned cells',
                    cleaned / len(self.dirty))

#Función encargada de animar la gráfica
def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('type_agent')
    color_dict = {0:'#5AD321', 1:'#330AC1', 2:'#EFF4F8', None:'#ffffff'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Robots Cleaning a Room\n"
                 f"Time-step: {model.t}, Dirty Cells left: "
                 f"{len(model.dirty.select(model.dirty.type_agent == 1))}")

fig, ax = plt.subplots()
model = DirtyRoom(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))